### Подгрузка необходимых библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time 
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

## Часть 1. Парсинг

### Парсер для ссылок на книги

In [2]:
links = []
basic_link = 'https://www.litres.ru'
for i in tqdm(np.arange(1, 120), desc = 'Loading...'):
    response = requests.get(f"https://www.litres.ru/genre/programmirovanie-5272/?page={i}")
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    basic_link = 'https://www.litres.ru'
    obj = soup.find_all('div', attrs = {'class':'ArtInfo_wrapper__GoMsb'})
    time.sleep(1)
    for i in np.arange(len(obj)):
        link = basic_link + obj[i].find('a').get('href')
        links.append(link)

Loading...: 100%|█████████████████████████████| 119/119 [02:01<00:00,  1.02s/it]


### Датафрейм для сохранения данных

In [89]:
data = pd.DataFrame(columns=['name', 'author', 'link',
                            'rating', 'rating_count','review_count',
                            'pages_count', 'price',
                            'text_reviews', 'age', 'year'])

### Парсер данных

In [90]:
missed_books_index = []
for i in tqdm(np.arange(len(links))):
    try:
        book_link = links[i]                      
        response = requests.get(book_link)
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        name = soup.find('h1', attrs = {'class':'BookCard_book__mainInfo__title___nFS4'}).text
        auth = soup.find('div', attrs = {'class':'Truncate_truncated__jKdVt'}).find('span').text
        link = book_link
        rating = float(soup
         .find_all('div', class_=re.compile('rating'))[0]
         .find('meta',attrs = {'itemprop':'ratingValue'}).get('content'))
        r_count = int(soup
         .find_all('div', class_=re.compile('rating'))[0]
         .find('meta',attrs = {'itemprop':'ratingCount'}).get('content'))
        try:
            rev_c = int(soup.find('span', attrs = {'class': 'CommentSystem_commentsCount__woGh3'}).text)
        except:
            rev_c = 0
        p_c = int(soup.find('div', attrs = {'class': 'BookCard_book__preview__data__XjF_j'}).find_all('p')[0].text.split(' ')[1])
        if len(soup.find_all('div', class_='SaleBlock_block__price__pC6XL')) == 1:
            price = soup.find_all('div', class_='SaleBlock_block__price__pC6XL')[0].text
        else:
            price = soup.find_all('div', class_='SaleBlock_block__price__pC6XL')[1].text
        obj = soup.find_all('div', attrs = {'class':'Comment_reviewText__PEkHn'})
        text_r = []
        for j in np.arange(len(obj)):
            text_r.append(obj[j].text)
        age = soup.find_all('div', attrs = {'class': 'CharacteristicsBlock_characteristic__4pi7v'})[0].find_all('span')[1].text.strip('+')
        obj1 = soup.find_all('div', attrs = {'class': 'CharacteristicsBlock_characteristic__4pi7v'})
        if obj1[2].find_all('span')[0].text == 'Дата написания: ':
            year = obj1[2].find_all('span')[1].text
        elif obj1[3].find_all('span')[0].text == 'Дата написания: ':
            year = obj1[3].find_all('span')[1].text
        else:
            year = None
        row = [name, auth, link, rating, r_count, rev_c,
              p_c, price, text_r, age, year]
        data.loc[i] = row
        time.sleep(1)
    except:
        missed_books_index.append(i)

100%|█████████████████████████████████████| 2851/2851 [1:14:43<00:00,  1.57s/it]


In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2534 entries, 0 to 2850
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          2534 non-null   object 
 1   author        2534 non-null   object 
 2   link          2534 non-null   object 
 3   rating        2534 non-null   float64
 4   rating_count  2534 non-null   int64  
 5   review_count  2534 non-null   int64  
 6   pages_count   2534 non-null   int64  
 7   price         2534 non-null   object 
 8   text_reviews  2534 non-null   object 
 9   age           2534 non-null   object 
 10  year          1976 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 237.6+ KB


In [93]:
data1 = data.copy()

In [111]:
data1.price = (data.price.str
               .replace('\xa0₽', '', regex=True)
               .replace(',', '.', regex=True)
               .replace('\xa0', '', regex=True)
               .replace('599.20749−20%', '599.20', regex=True)
               .replace('575.10639−10%', '640.0', regex=True)
              .astype(float))

In [113]:
data1.reset_index(inplace=True)

In [114]:
data1.to_csv('litresdata.csv')

## Часть 2. EDA

In [36]:
df = pd.read_csv('litresdata.csv', index_col='index')
df = df.drop(columns=['Unnamed: 0'])
df.reset_index(inplace=True, drop=True)

1. Выведите первые 5 строк датасета. (0.25). Сколько в нём строк и столбцов (0.25)?

In [37]:
print(df.shape)
df.head()

(2534, 11)


,name,author,link,rating,rating_count,review_count,pages_count,price,text_reviews,age,year
0,Изучаем DDD – предметно-ориентированное проект...,Влад Хононов,https://www.litres.ru/book/vlad-hononov/izucha...,0.0,0,1,319,549.0,"['Обязательно вернусь, но позжеЭта книга из то...",0,2022.0
1,Высоконагруженные приложения. Программирование...,Мартин Клеппман,https://www.litres.ru/book/martin-kleppman-173...,4.0,51,11,640,799.0,"['Не могу что-либо про перевод сказать, читал ...",16,2017.0
2,Начинаем программировать на Python,Тони Гэддис,https://www.litres.ru/book/toni-geddis-3249116...,5.0,16,6,875,599.0,['Шикарная книга! Начал изучать Python самосто...,12,2021.0
3,System Design. Подготовка к сложному интервью,Алекс Сюй,https://www.litres.ru/book/aleks-suy/system-de...,3.6,17,9,304,699.0,['Добротная книга по разработке архитектуры пр...,16,2020.0
4,Нейросети начало,Джейд Картер,https://www.litres.ru/book/dzheyd-karter/neyro...,4.9,439,37,102,690.0,['Книга отличная. Давно хотела понять как все ...,12,2023.0


В датасете имеется 2534 строки и 11 стобцов

2. Есть ли в датасете пропуски? (0.5)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          2534 non-null   object 
 1   author        2534 non-null   object 
 2   link          2534 non-null   object 
 3   rating        2534 non-null   float64
 4   rating_count  2534 non-null   int64  
 5   review_count  2534 non-null   int64  
 6   pages_count   2534 non-null   int64  
 7   price         2534 non-null   float64
 8   text_reviews  2534 non-null   object 
 9   age           2534 non-null   object 
 10  year          1976 non-null   float64
dtypes: float64(3), int64(3), object(5)
memory usage: 217.9+ KB


In [39]:
df.isna().sum()

name              0
author            0
link              0
rating            0
rating_count      0
review_count      0
pages_count       0
price             0
text_reviews      0
age               0
year            558
dtype: int64

В датасете имеется 558 пропусков с данными по году написания

3.Проверьте типы данных. Если это необходимо, приведите к типам int и float те столбцы, с которыми понадобится работать как с числами. (1).

In [40]:
df.dtypes

name             object
author           object
link             object
rating          float64
rating_count      int64
review_count      int64
pages_count       int64
price           float64
text_reviews     object
age              object
year            float64
dtype: object

In [41]:
df.age = (df.age.str.replace('26 из ~57', '12', regex=True)
         .replace('670 из ~150', '12', regex=True))

In [48]:
df.age = df.age.astype(str)
df.year = df.year.astype(str)

4. Выведите описательные статистики переменных. Ответьте на следующие вопросы:
- Какая медианная цена книги в вашем датасете? (1)
- Какое возрастное ограничение встречается чаще всего? (1)
- Какое среднее число отзывов в книге? (1)
- Сколько книг имеют оценку ниже 4.25? (1)
- В каком году было написано больше всего книг из датасета? (1)

In [51]:
df.describe()

,rating,rating_count,review_count,pages_count,price
count,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000
mean,2.481018,7.106551,1.242699,335.012628,1425.752135
std,2.127763,25.894478,2.811750,260.384821,3218.356614
min,0.000000,0.000000,0.000000,2.000000,40.000000
25%,0.000000,0.000000,0.000000,129.000000,168.000000
50%,3.400000,1.000000,0.000000,313.000000,389.000000
75%,4.500000,5.000000,1.000000,466.000000,699.000000
max,5.000000,439.000000,41.000000,1588.000000,31687.580000


In [50]:
df[['author', 'age','year']].describe(include='object')

,author,age,year
count,2534,2534,2534
unique,1811,5,31
top,Владимир Дронов,0,nan
freq,27,1934,558


In [55]:
df.author.value_counts()[:5]

Владимир Дронов      27
Никита Культин       26
Джейд Картер         16
Тимур Машнин         15
коллектив авторов    15
Name: author, dtype: int64

In [56]:
df.age.value_counts()[:5]

0     1934
12     299
16     255
6       40
18       6
Name: age, dtype: int64

In [57]:
df.year.value_counts()[:5]

nan       558
2021.0    170
2023.0    149
2022.0    130
2020.0    125
Name: year, dtype: int64

In [58]:
(df.rating < 4.25).sum()

1765

- Медианная цена: 389
- Наиболе часто встречается ограничение: 0+
- Среднее число отзывов: 1.242699
- 1765 книг имееют рейтинг ниже 4.25
- В 2021 году написано набольшее количество книг из датасета

5. Если вы работаете с готовым датасетом, то попробуйте "достать" из столбца pages количество страниц. Если у вас не получилось, то далее при определении числа страниц пользуйтесь стольцов pages_count. Если вы парсили датасет сами, то вы получаете балл за этот пункт автоматически (1.5)

Датасет парсился

6. Создайте новое поле is_popular. Значение равно 1, если рейтинг книги не менее 4.6 и при этом у нее не менее 5 отзывов, и 0 в остальных случаях. (1)

In [63]:
df['is_popular'] = (df.rating > 4.6)&(df.review_count >= 5)
df.is_popular = df.is_popular.astype(int)

7. Как отличается среднее число страниц среди популярных и непопулярных книг? (2)

In [71]:
a = df[df.is_popular == 1].pages_count.mean()

In [72]:
b = df[df.is_popular == 0].pages_count.mean()

In [75]:
round((a/b - 1)*100)

8

У популярных книг на 8% больше страниц. Однако их число в датасете крайне малочисленно

8. Выведите топ-10 книг по числу отзывов. (2).

In [81]:
df.sort_values(by = 'review_count', ascending=False)[['name']].head(10)

,name
419,Оптимизация в Python
4,Нейросети начало
200,Психбольница в руках пациентов. Алан Купер об ...
120,Теоретический минимум по Computer Science. Все...
327,Идеальный программист. Как стать профессионало...
450,"Говорят, в IT много платят. Как построить успе..."
146,#Сам себе программист. Как научиться программи...
39,Сделай видеоигру один и не свихнись
56,Как пасти котов. Наставление для программистов...
141,Экстремальное программирование. Разработка чер...


9. Найдите среднюю длину отзыва (в символах). 
(Не смог сделать, т.к. не собрал все возможные отзывы)

10. Постройте таблицу корреляций числовых переменных. (1) Прокомментируйте результаты. (1)

In [82]:
df.corr()

/var/folders/yp/blf1pq316cv6y1tkcp2lpn0c0000gn/T/ipykernel_44806/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,rating,rating_count,review_count,pages_count,price,is_popular
rating,1.000000,0.238950,0.350593,0.189762,-0.278476,0.129759
rating_count,0.238950,1.000000,0.687595,0.037633,-0.073984,0.606725
review_count,0.350593,0.687595,1.000000,0.089686,-0.126599,0.388756
pages_count,0.189762,0.037633,0.089686,1.000000,0.227830,0.011493
price,-0.278476,-0.073984,-0.126599,0.227830,1.000000,-0.028902
is_popular,0.129759,0.606725,0.388756,0.011493,-0.028902,1.000000


В датасете не обнаружено особо сильных корреляций числовых переменных помимо очевидных (таких как корреляция количества оценок и отзывов). Из забавного, цена отрицательно коррелирует с рейтингом.

На этом с заданиями все, так как не успеваю к дедлайну. Позже обновлю страницу доделанным заданием, но уже просто для себя)))